# Lab 4: Design and Query OLTP and OLAP Systems (PostgreSQL, Google Colab Version)

In this Colab version, we:

- Install and start PostgreSQL directly inside the Colab runtime
- Configure the `postgres` user so Python can connect with a password
- Use Python (`psycopg2`) to send SQL commands to PostgreSQL
- Keep **all major steps** of the lab:
  1. Set up PostgreSQL and connect to the database  
  2. Implement an **OLTP** (normalized) schema  
  3. Implement an **OLAP** (star) schema  
  4. Populate both schemas with synthetic data  
  5. Compare **OLTP vs. OLAP** query performance using `EXPLAIN ANALYZE`

> 💡 **Goal:** Experience how OLTP (transactional) and OLAP (analytical) designs differ in structure and performance, using the same underlying data.


## Task 1 – Setting up PostgreSQL and Connecting to the Database

### Original Lab Instructions (for reference)


In this **Google Colab**, we:

1. Install PostgreSQL and its tools directly in the Colab environment  
2. Start the PostgreSQL service  
3. Set a password for the `postgres` user  
4. Connect from Python using `psycopg2`

Run the following cell **once** at the top of your session to set up PostgreSQL.


In [1]:
# Task 1 – Install and start PostgreSQL, and install Python dependencies
# This may take a few minutes the first time it runs.

!apt-get -y update
!apt-get -y install postgresql postgresql-contrib

# Start the PostgreSQL service
!service postgresql start

# Set a password for the 'postgres' superuser so we can connect via TCP
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"

# Install psycopg2 so Python can talk to PostgreSQL
!pip install -q psycopg2-binary

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/u

### Connect to PostgreSQL from Python

Below we:

1. Connect to the default `postgres` database as the `postgres` superuser  
2. Drop and recreate a clean `query_demo` database  
3. Connect to `query_demo`  
4. Define a helper function `run_sql` to send SQL commands and optionally print results

> 🔁 If you re-run this notebook from the top, the database will be recreated fresh each time.


In [2]:
import psycopg2

# Connect to the default 'postgres' database to manage databases.
# We connect over TCP to 127.0.0.1 using the password we just set.
admin_conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="127.0.0.1",
    port=5432
)
admin_conn.autocommit = True
admin_cur = admin_conn.cursor()

# Drop and recreate the lab database to ensure a clean environment
admin_cur.execute("DROP DATABASE IF EXISTS query_demo;")
admin_cur.execute("CREATE DATABASE query_demo;")

admin_cur.close()
admin_conn.close()

# Connect to the lab database
conn = psycopg2.connect(
    dbname="query_demo",
    user="postgres",
    password="postgres",
    host="127.0.0.1",
    port=5432
)
conn.autocommit = True
cur = conn.cursor()

def run_sql(sql, fetch=False, max_rows=20):
    """Helper to run SQL against PostgreSQL and optionally print results.

    Args:
        sql (str): SQL command(s) to execute.
        fetch (bool): If True, fetch and print results.
        max_rows (int): Max number of rows to display when fetching.
    """
    print("Running SQL:\n" + "-" * 80)
    print(sql.strip())
    print("-" * 80)
    cur.execute(sql)

    if fetch:
        rows = cur.fetchall()
        print(f"Returned {len(rows)} rows (showing up to {max_rows}):")
        for row in rows[:max_rows]:
            print(row)
    else:
        try:
            print(f"Rowcount: {cur.rowcount}")
        except Exception:
            pass

    print("\n")

## Task 2 – Implementing an OLTP System (Normalized Schema)

In an **OLTP** (Online Transaction Processing) system, the schema is typically **normalized** to:

- Reduce data redundancy  
- Maintain data integrity (through foreign keys)  
- Support many small, frequent **INSERT/UPDATE/DELETE** operations  

In this lab, the normalized OLTP schema consists of:

- **`customers`** – one row per customer  
- **`products`** – one row per product  
- **`orders`** – one row per order (header)  
- **`order_items`** – one row per line item on an order  

We will now create these tables exactly as in the original lab.


In [3]:
# Task 2 – Create OLTP (normalized) tables
create_oltp_sql = """
DROP TABLE IF EXISTS order_items CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS products CASCADE;
DROP TABLE IF EXISTS customers CASCADE;

CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100) UNIQUE
);

CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    price DECIMAL(10,2)
);

CREATE TABLE orders (
    order_id SERIAL PRIMARY KEY,
    customer_id INT REFERENCES customers(customer_id),
    order_date TIMESTAMP DEFAULT NOW()
);

CREATE TABLE order_items (
    order_item_id SERIAL PRIMARY KEY,
    order_id INT REFERENCES orders(order_id) ON DELETE CASCADE,
    product_id INT REFERENCES products(product_id),
    quantity INT,
    price DECIMAL(10,2)
);
"""

run_sql(create_oltp_sql)

Running SQL:
--------------------------------------------------------------------------------
DROP TABLE IF EXISTS order_items CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS products CASCADE;
DROP TABLE IF EXISTS customers CASCADE;

CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100) UNIQUE
);

CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    price DECIMAL(10,2)
);

CREATE TABLE orders (
    order_id SERIAL PRIMARY KEY,
    customer_id INT REFERENCES customers(customer_id),
    order_date TIMESTAMP DEFAULT NOW()
);

CREATE TABLE order_items (
    order_item_id SERIAL PRIMARY KEY,
    order_id INT REFERENCES orders(order_id) ON DELETE CASCADE,
    product_id INT REFERENCES products(product_id),
    quantity INT,
    price DECIMAL(10,2)
);
--------------------------------------------------------------------------------
Rowcount: -1




## Task 3 – Implementing an OLAP System (Star Schema)

An **OLAP** (Online Analytical Processing) system is optimized for **analytics and reporting**, not for frequent small transactions.

A common OLAP design is the **star schema**, consisting of:

- A central **fact table** with metrics (e.g. `fact_sales`)  
- Several surrounding **dimension tables** with descriptive attributes (e.g. `dim_customers`, `dim_products`)  

In this lab, the OLAP schema includes:

- **`dim_customers`** – dimension table with customer information  
- **`dim_products`** – dimension table with product information  
- **`fact_sales`** – fact table with sales transactions (quantities and prices)  

We will now create these star-schema tables.


In [4]:
# Task 3 – Create OLAP (star schema) tables
create_olap_sql = """
DROP TABLE IF EXISTS fact_sales CASCADE;
DROP TABLE IF EXISTS dim_products CASCADE;
DROP TABLE IF EXISTS dim_customers CASCADE;

CREATE TABLE dim_customers (
    customer_key SERIAL PRIMARY KEY,
    customer_id INT UNIQUE,
    name VARCHAR(100),
    email VARCHAR(100)
);

CREATE TABLE dim_products (
    product_key SERIAL PRIMARY KEY,
    product_id INT UNIQUE,
    name VARCHAR(100),
    price DECIMAL(10,2)
);

CREATE TABLE fact_sales (
    sale_key SERIAL PRIMARY KEY,
    customer_key INT REFERENCES dim_customers(customer_key),
    product_key INT REFERENCES dim_products(product_key),
    sale_date TIMESTAMP DEFAULT NOW(),
    quantity INT,
    total_price DECIMAL(10,2)
);
"""

run_sql(create_olap_sql)

Running SQL:
--------------------------------------------------------------------------------
DROP TABLE IF EXISTS fact_sales CASCADE;
DROP TABLE IF EXISTS dim_products CASCADE;
DROP TABLE IF EXISTS dim_customers CASCADE;

CREATE TABLE dim_customers (
    customer_key SERIAL PRIMARY KEY,
    customer_id INT UNIQUE,
    name VARCHAR(100),
    email VARCHAR(100)
);

CREATE TABLE dim_products (
    product_key SERIAL PRIMARY KEY,
    product_id INT UNIQUE,
    name VARCHAR(100),
    price DECIMAL(10,2)
);

CREATE TABLE fact_sales (
    sale_key SERIAL PRIMARY KEY,
    customer_key INT REFERENCES dim_customers(customer_key),
    product_key INT REFERENCES dim_products(product_key),
    sale_date TIMESTAMP DEFAULT NOW(),
    quantity INT,
    total_price DECIMAL(10,2)
);
--------------------------------------------------------------------------------
Rowcount: -1




## Task 4 – Populating OLTP and OLAP Databases

To compare OLTP and OLAP performance meaningfully, we need **lots of data**.

We will populate the database as follows:

1. Insert **10 customers** with synthetic names and emails  
2. Insert **5 products** with fixed names and prices  
3. Insert **100,000 orders** with:
   - Random customers  
   - Random order dates within the last year  
4. Insert **100,000 order items** with:
   - Random product  
   - Random quantity  
   - Random price  
5. Populate the OLAP tables by:
   - Copying customers into `dim_customers`  
   - Copying products into `dim_products`  
   - Creating `fact_sales` rows from the OLTP `orders` and `order_items` tables  

The synthetic data is generated **inside PostgreSQL** using `generate_series` and `random()`.


### 4.1 Insert Sample Customers

We create 10 synthetic customers with names like `Customer_1`, `Customer_2`, etc., and matching email addresses.


In [5]:
insert_customers_sql = """
INSERT INTO customers (name, email)
SELECT 'Customer_' || i, 'customer' || i || '@example.com'
FROM generate_series(1, 10) AS i;
"""

run_sql(insert_customers_sql)

Running SQL:
--------------------------------------------------------------------------------
INSERT INTO customers (name, email) 
SELECT 'Customer_' || i, 'customer' || i || '@example.com'
FROM generate_series(1, 10) AS i;
--------------------------------------------------------------------------------
Rowcount: 10




### 4.2 Insert Sample Products

We insert a small product catalog of 5 products with fixed prices.


In [6]:
insert_products_sql = """
INSERT INTO products (name, price)
VALUES ('Laptop', 1200.00),
       ('Mouse', 25.00),
       ('Keyboard', 50.00),
       ('Monitor', 300.00),
       ('Phone', 800.00);
"""

run_sql(insert_products_sql)

Running SQL:
--------------------------------------------------------------------------------
INSERT INTO products (name, price) 
VALUES ('Laptop', 1200.00),
       ('Mouse', 25.00),
       ('Keyboard', 50.00),
       ('Monitor', 300.00),
       ('Phone', 800.00);
--------------------------------------------------------------------------------
Rowcount: 5




### 4.3 Insert 100,000 Orders

Each order:

- Is assigned to a **random customer** from 1 to 10  
- Gets an `order_date` randomly distributed over the last 365 days  

> ⚠️ This step may take a little time, since it creates 100,000 rows.


In [7]:
insert_orders_sql = """
INSERT INTO orders (customer_id, order_date)
SELECT (random() * 9 + 1)::INT, NOW() - (random() * INTERVAL '365 days')
FROM generate_series(1, 100000);
"""

run_sql(insert_orders_sql)

Running SQL:
--------------------------------------------------------------------------------
INSERT INTO orders (customer_id, order_date)
SELECT (random() * 9 + 1)::INT, NOW() - (random() * INTERVAL '365 days')
FROM generate_series(1, 100000);
--------------------------------------------------------------------------------
Rowcount: 100000




### 4.4 Insert 100,000 Order Items

Each order item row:

- Is linked to an existing `orders.order_id`  
- Uses a random `product_id` from 1 to 5  
- Has a random quantity from 1 to 4  
- Has a random price between 20 and 320  

> Again, this may take a little time because of the volume of data.


In [8]:
insert_order_items_sql = """
INSERT INTO order_items (order_id, product_id, quantity, price)
SELECT o.order_id,
       (random() * 4 + 1)::INT,
       (random() * 3 + 1)::INT,
       (random() * 300 + 20)::NUMERIC
FROM orders o
ORDER BY random()
LIMIT 100000;
"""

run_sql(insert_order_items_sql)

Running SQL:
--------------------------------------------------------------------------------
INSERT INTO order_items (order_id, product_id, quantity, price)
SELECT o.order_id,
       (random() * 4 + 1)::INT,
       (random() * 3 + 1)::INT,
       (random() * 300 + 20)::NUMERIC
FROM orders o
ORDER BY random()
LIMIT 100000;
--------------------------------------------------------------------------------
Rowcount: 100000




### 4.5 Populate OLAP (Star Schema) Tables

Now we:

1. Copy customers from `customers` → `dim_customers`  
2. Copy products from `products` → `dim_products`  
3. Create fact rows in `fact_sales` by joining `orders`, `order_items`, `dim_customers`, and `dim_products`  

This simulates a typical **ETL (Extract–Transform–Load)** process from OLTP into OLAP.


In [9]:
populate_olap_sql = """
-- 1. Populate dim_customers
INSERT INTO dim_customers (customer_id, name, email)
SELECT customer_id, name, email
FROM customers;

-- 2. Populate dim_products
INSERT INTO dim_products (product_id, name, price)
SELECT product_id, name, price
FROM products;

-- 3. Populate fact_sales
INSERT INTO fact_sales (customer_key, product_key, sale_date, quantity, total_price)
SELECT c.customer_key,
       p.product_key,
       o.order_date,
       oi.quantity,
       (oi.price * oi.quantity)
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN dim_customers c ON o.customer_id = c.customer_id
JOIN dim_products p ON oi.product_id = p.product_id;
"""

run_sql(populate_olap_sql)

Running SQL:
--------------------------------------------------------------------------------
-- 1. Populate dim_customers
INSERT INTO dim_customers (customer_id, name, email)
SELECT customer_id, name, email
FROM customers;

-- 2. Populate dim_products
INSERT INTO dim_products (product_id, name, price)
SELECT product_id, name, price
FROM products;

-- 3. Populate fact_sales
INSERT INTO fact_sales (customer_key, product_key, sale_date, quantity, total_price)
SELECT c.customer_key,
       p.product_key,
       o.order_date,
       oi.quantity,
       (oi.price * oi.quantity)
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN dim_customers c ON o.customer_id = c.customer_id
JOIN dim_products p ON oi.product_id = p.product_id;
--------------------------------------------------------------------------------
Rowcount: 100000




### 4.6 Sanity Check – Row Counts

Let’s quickly confirm that the tables contain data as expected.


In [10]:
for table in ["customers", "products", "orders", "order_items",
               "dim_customers", "dim_products", "fact_sales"]:
    run_sql(f"SELECT COUNT(*) FROM {table};", fetch=True, max_rows=5)

Running SQL:
--------------------------------------------------------------------------------
SELECT COUNT(*) FROM customers;
--------------------------------------------------------------------------------
Returned 1 rows (showing up to 5):
(10,)


Running SQL:
--------------------------------------------------------------------------------
SELECT COUNT(*) FROM products;
--------------------------------------------------------------------------------
Returned 1 rows (showing up to 5):
(5,)


Running SQL:
--------------------------------------------------------------------------------
SELECT COUNT(*) FROM orders;
--------------------------------------------------------------------------------
Returned 1 rows (showing up to 5):
(100000,)


Running SQL:
--------------------------------------------------------------------------------
SELECT COUNT(*) FROM order_items;
--------------------------------------------------------------------------------
Returned 1 rows (showing up to 5):
(100000

## Task 5 – Comparing OLTP vs. OLAP Query Performance

Now that we have large datasets in both schemas, we can compare query performance.

We’ll use `EXPLAIN ANALYZE` to measure the execution plan and timing for:

1. An aggregation query on the **normalized OLTP schema** (joins across `orders`, `customers`, `order_items`, `products`)  
2. An aggregation query on the **OLAP star schema** (joins across `fact_sales`, `dim_customers`, `dim_products`)  

In both cases, we compute:

- Total quantity per (customer, product)  
- Total revenue per (customer, product)  

> 💡 Watch how the execution time and join complexity differ between the two designs.


### 5.1 OLTP Query Performance (Normalized Schema)

This query uses the **normalized** tables directly:

```sql
SELECT c.name, p.name, SUM(oi.quantity) AS total_quantity,
       SUM(oi.price * oi.quantity) AS total_revenue
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY c.name, p.name;
```

We’ll wrap it with `EXPLAIN ANALYZE` to see the execution plan and timing.


In [11]:
oltp_explain_sql = """
EXPLAIN ANALYZE
SELECT c.name, p.name, SUM(oi.quantity) AS total_quantity,
       SUM(oi.price * oi.quantity) AS total_revenue
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY c.name, p.name;
"""

run_sql(oltp_explain_sql, fetch=True, max_rows=50)  # show first 50 lines of the plan

Running SQL:
--------------------------------------------------------------------------------
EXPLAIN ANALYZE
SELECT c.name, p.name, SUM(oi.quantity) AS total_quantity, 
       SUM(oi.price * oi.quantity) AS total_revenue
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY c.name, p.name;
--------------------------------------------------------------------------------
Returned 23 rows (showing up to 50):
('GroupAggregate  (cost=35682.15..37857.15 rows=34000 width=476) (actual time=236.459..288.415 rows=50 loops=1)',)
('  Group Key: c.name, p.name',)
('  ->  Sort  (cost=35682.15..35932.15 rows=100000 width=446) (actual time=235.808..256.730 rows=100000 loops=1)',)
('        Sort Key: c.name, p.name',)
('        Sort Method: external merge  Disk: 4032kB',)
('        ->  Hash Join  (cost=3212.57..7210.33 rows=100000 width=446) (actual time=27.986..141.461 rows=100000 loops=

### 5.2 OLAP Query Performance (Star Schema)

Now we run the equivalent aggregation query against the **OLAP star schema**:

```sql
SELECT c.name, p.name, SUM(f.quantity) AS total_quantity,
       SUM(f.total_price) AS total_revenue
FROM fact_sales f
JOIN dim_customers c ON f.customer_key = c.customer_key
JOIN dim_products p ON f.product_key = p.product_key
GROUP BY c.name, p.name;
```

Again, we wrap it in `EXPLAIN ANALYZE` to inspect the plan and performance.


In [12]:
olap_explain_sql = """
EXPLAIN ANALYZE
SELECT c.name, p.name, SUM(f.quantity) AS total_quantity,
       SUM(f.total_price) AS total_revenue
FROM fact_sales f
JOIN dim_customers c ON f.customer_key = c.customer_key
JOIN dim_products p ON f.product_key = p.product_key
GROUP BY c.name, p.name;
"""

run_sql(olap_explain_sql, fetch=True, max_rows=50)  # show first 50 lines of the plan

Running SQL:
--------------------------------------------------------------------------------
EXPLAIN ANALYZE
SELECT c.name, p.name, SUM(f.quantity) AS total_quantity, 
       SUM(f.total_price) AS total_revenue
FROM fact_sales f
JOIN dim_customers c ON f.customer_key = c.customer_key
JOIN dim_products p ON f.product_key = p.product_key
GROUP BY c.name, p.name;
--------------------------------------------------------------------------------
Returned 18 rows (showing up to 50):
('GroupAggregate  (cost=30871.64..32546.64 rows=34000 width=476) (actual time=149.989..184.415 rows=50 loops=1)',)
('  Group Key: c.name, p.name',)
('  ->  Sort  (cost=30871.64..31121.64 rows=100000 width=446) (actual time=149.290..168.086 rows=100000 loops=1)',)
('        Sort Key: c.name, p.name',)
('        Sort Method: external merge  Disk: 4032kB',)
('        ->  Hash Join  (cost=30.57..2399.82 rows=100000 width=446) (actual time=0.040..40.524 rows=100000 loops=1)',)
('              Hash Cond: (f.product_key

## Performance Summary and Reflection

In general, you should observe something like:

| Aspect                 | OLTP (Normalized Schema)                     | OLAP (Star Schema)                          |
|------------------------|----------------------------------------------|---------------------------------------------|
| Query speed            | Slower for large aggregations                | Faster for analytics                        |
| JOINs required         | Many (e.g., 4+ tables)                       | Fewer (2–3 tables)                          |
| Best for               | Transactions (INSERTs, UPDATEs, small reads) | Reporting, BI, aggregations                 |
| Storage                | More efficient (less redundancy)             | Larger (pre-aggregated / duplicated data)   |

### Lab Review Questions

1. **Which system (OLTP or OLAP) performed faster for aggregations in your run? Why do you think that is?**  
2. **How would indexing affect OLTP query performance?**  
   - A. Improve performance for reads  
   - B. Slow down inserts  
   - C. No impact  
   - D. Both A and B  

Think about the trade-offs between **write performance**, **read performance**, and **storage** when choosing between normalized and star schemas in real-world systems.


### (Optional) Close the Database Connection

When you’re completely done with the lab, you can close the PostgreSQL connection.


In [13]:
# Optional: close the connection when finished
cur.close()
conn.close()
print("PostgreSQL connection closed.")

PostgreSQL connection closed.
